In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import logging
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

## Loding data

In [ ]:
notes = pd.read_csv('notes.csv')[['id', 'text']]
print(f'shape of the notes file: {notes.shape}')
n_sub = len(pd.unique(notes['id']))
print(f'unique number of subjects: {n_sub}')
notes.head()

In [ ]:
# distribution of how many notes each subject has
notes['id'].value_counts().describe()

In [ ]:
notes['text'] = notes['text'].str.replace("___", "")
notes['text'] = notes['text'].str.lower()
notes.head()

### loading essential functions

In [ ]:
def token_distribution(df, model_name):
  # load tokenizer
  tokenizer = BertTokenizer.from_pretrained(model_name)
  tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  # investigate token
  rand_id = random.randint(0, len(tokenized)-1)
  print(f'random id: {rand_id}')
  print(df['text'][rand_id])
  print(tokenizer.convert_ids_to_tokens(tokenized[rand_id]))
  print(f'length of the token: {len(tokenized[rand_id])}')


  # visualize token distribution
  max_len = 0
  n = 0
  len_ls = tokenized.apply(lambda x: len(x))
  for i in tokenized.values:
    if len(i) > 512:
      n+=1
    if len(i) > max_len:
        max_len = len(i)
  print(f'maximum lenght is {max_len}')
  print(f'{n} out of {df.shape[0]} ({n/df.shape[0]*100} %) notes exceed the maximum embedding length of 512')

  print(len_ls.describe())
  ax = len_ls.plot.box()
  plt.axhline(y=512, color='r', linestyle='--')
  plt.show()


def obtain_batch_embedding(df, model_name):

  # load tokenizer
  tokenizer = BertTokenizer.from_pretrained(model_name)
  tokenizer.truncation_side='left' # truncate from left to preserve more findings

  # obtain token id and attention mask
  input_ids = []
  attention_masks = []
  for t in df['text']:
    encoded_dict = tokenizer.encode_plus(t,
                                        add_special_tokens=True,
                                        max_length=512,
                                        padding='max_length',
                                        return_attention_mask=True,
                                        return_tensors='pt',
                                        truncation=True)
    input_ids.append(encoded_dict['input_ids'].to(device))
    attention_masks.append(encoded_dict['attention_mask'].to(device))

  # convert to tensor
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  print("tokens obtained, start fitting model")

  # fit model
  model = BertModel.from_pretrained(model_name)
  model = model.to(device)
  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_masks)

  features = pd.DataFrame(last_hidden_states[0][:,0,:].cpu().numpy())
  features.reset_index(drop=True, inplace=True)
  # print(df['id'].shape, features.shape)

  # concatenate the id column
  res = pd.concat([df['id'].reset_index(drop=True), features], axis=1, ignore_index=True)
  print(res.shape)

  # take mean value of the embeddings for each individual
  # res = features.groupby('id').mean()

  return res

def embedding(df, model_name, batch_size=256):

  # loop through batches
  embeddings = pd.DataFrame()
  for i in range(0, df.shape[0], batch_size):
    print(f'batch from {i}th to {i+batch_size}th sample')
    batch = df.iloc[i:i+batch_size]
    batch_embedding = obtain_batch_embedding(batch, model_name)
    embeddings = pd.concat([embeddings, batch_embedding], axis=0)

  return embeddings


In [ ]:
notes['id'].reset_index(drop=True).head()

### set up device

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

### generating embedding

In [ ]:
# to investigate the distribution of the tokens
token_distribution(notes, "neuml/pubmedbert-base-embeddings")

In [ ]:
# obtain embeddings
res = embedding(notes, "neuml/pubmedbert-base-embeddings")
print(res.shape)
res.head()

In [ ]:
res.to_csv('pubmedbert_embedding.csv', index=False)

## Embedding aggregation
1. average
2. max pooling
3. randomly select one

In [ ]:
embedding = pd.read_csv("./pubmedbert_embedding.csv")
embedding.rename(columns={'0': 'id'}, inplace=True)
print(embedding.shape)
embedding.head()

In [ ]:
test = embedding[20000:22000]
test.rename(columns={'0': 'id'}, inplace=True)
test.head()

In [ ]:
pd.DataFrame(test[test['id']==20214994].iloc[1]).T

In [ ]:
pd.unique(test['id'])

In [ ]:
def avg(df):
    res = df.groupby('id').mean().reset_index()
    return res

def max_pooling(df):
    res = df.groupby('id').max().reset_index()
    return res

def return_random_note(rows, rand_n):
    # rows = df[df['id']==id]
    # print(rows.shape, rand_n)
    res = pd.DataFrame(rows.iloc[rand_n]).T
    return res

def random_sample(df):
    # obtain count of notes for each id
    N = df.groupby('id').size().to_frame().reset_index()
    N.rename(columns={0: 'count'}, inplace=True)

    # generate a random number for each id
    N['rand'] = N.apply(lambda x: np.random.randint(0, x['count']), axis=1)

    # obtain random note for each id
    res = pd.DataFrame()
    count = 0
    id_ls = pd.unique(df['id'])
    res_ls = []
    for id in id_ls:
        rand_n = N[N['id']==id]['rand'].iloc[0]
        rows = df[df['id']==id]
        selected = return_random_note(rows, rand_n)
        # res = pd.concat([res, selected], ignore_index=True)
        res_ls.append(selected)

        count += 1
        if count % 500 == 0:
            print(f"{count} unique ids finished")

    res = pd.concat(res_ls)
    return res

In [ ]:
pubmed_avg = avg(embedding)
print(pubmed_avg.shape)
# pubmed_avg.to_csv("pubmed_avg.csv", index=False)# 

In [ ]:
pubmed_max = max_pooling(embedding)
print(pubmed_max.shape)
# pubmed_max.to_csv("pubmed_max.csv", index=False)

In [ ]:
del pubmed_avg
del pubmed_max

In [ ]:
pd.unique(embedding['id']).shape

In [ ]:
embedding.columns

In [ ]:
pubmed_rand = random_sample(embedding)
print(pubmed_rand.shape)
pubmed_rand.head()
# pubmed_rand.to_csv("pubmed_rand.csv", index=False)

In [ ]:
pubmed_rand.to_csv("pubmed_rand.csv", index=False)